In [ ]:
# How to do correls over time without for loops:
# 1.) https://cablab.readthedocs.io/en/latest/dat_python.html - corrcf, but how to get p-val?
# same as above: https://github.com/esa-esdl/esdl-shared/blob/master/notebooks/Python/Python_DAT.ipynb
# 2.) http://xarray.pydata.org/en/stable/dask.html
import numpy as np
import xarray as xr
import bottleneck

def covariance_gufunc(x, y):
    return ((x - x.mean(axis=-1, keepdims=True))
            * (y - y.mean(axis=-1, keepdims=True))).mean(axis=-1)

def pearson_correlation_gufunc(x, y):
    return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))

def spearman_correlation_gufunc(x, y):
    x_ranks = bottleneck.rankdata(x, axis=-1)
    y_ranks = bottleneck.rankdata(y, axis=-1)
    return pearson_correlation_gufunc(x_ranks, y_ranks)

def spearman_correlation(x, y, dim):
    return xr.apply_ufunc(
        spearman_correlation_gufunc, x, y,
        input_core_dims=[[dim], [dim]],
        dask='parallelized',
        output_dtypes=[float])

In [19]: rs = np.random.RandomState(0)

In [20]: array1 = xr.DataArray(rs.randn(1000, 100000), dims=['place', 'time'])  # 800MB

In [21]: array2 = array1 + 0.5 * rs.randn(1000, 100000)

# using one core, on NumPy arrays
In [22]: %time _ = spearman_correlation(array1, array2, 'time')
CPU times: user 21.6 s, sys: 2.84 s, total: 24.5 s
Wall time: 24.9 s

In [23]: chunked1 = array1.chunk({'place': 10})

In [24]: chunked2 = array2.chunk({'place': 10})

# using all my laptop's cores, with Dask
In [25]: r = spearman_correlation(chunked1, chunked2, 'time').compute()

In [26]: %time _ = r.compute()
CPU times: user 30.9 s, sys: 1.74 s, total: 32.6 s
Wall time: 4.59 s

# OR THIS: https://examples.dask.org/xarray.html
corr = spearman_correlation(da.chunk({'time': -1}),
                            da_smooth.chunk({'time': -1}),
                            'time')
corr

# 3.) For loop methodology: https://stackoverflow.com/questions/45863969/python-how-to-find-regression-equation-of-multiple-3d-lat-lon-time-value-data 
for t in range(varA1['time'].size) :
    for la in range(varA1['lat'].size) :
        for lo in range(varA1['lon'].size) :
            x = varA1.values[t,la,lo]
            y = varB1.values[t,la,lo]
            plt.scatter(x,y)
for t in range(varA2['time'].size) :
    for la in range(varA2['lat'].size) :
        for lo in range(varA2['lon'].size) :
            x = varA2.values[t,la,lo]
            y = varB2.values[t,la,lo]
            plt.scatter(x,y)
... 
plt.show()
# OR THIS: https://stackoverflow.com/questions/38960903/applying-numpy-polyfit-to-xarray-dataset 
ds_LR = ds.TMP_P0_L103_GST0 * 0 -9999 # Quick way to make dataarray with -9999 values but with correct dims/coords
for cts in np.arange(0,len(ds_UA.time)):
        for cx in ds_UA.xgrid_0.values:
                for cy in ds_UA.ygrid_0.values:
                        x_temp = ds_UA.Temperature[:,cts,cy,cx] # Grab the vertical profile of air temperature
                        y_hgt  = ds_UA.Height[:,cts,cy,cx] # Grab the vertical heights of air temperature values
                        s      = np.polyfit(y_hgt,x_temp,1) # Fit a line to the data
                        ds_LR[cts,cy,cx].values = s[0] # Grab the slope (first element)
                        
#4.) THIS MAY BE THE ONE: https://stackoverflow.com/questions/52094320/with-xarray-how-to-parallelize-1d-operations-on-a-multidimensional-dataset
It is possible to apply scipy.stats.linregress (and other non-ufuncs) to the xarray Dataset using apply_ufunc() by passing vectorize=True like so:

# return a tuple of DataArrays
res = xr.apply_ufunc(scipy.stats.linregress, ds[x], ds[y],
        input_core_dims=[['year'], ['year']],
        output_core_dims=[[], [], [], [], []],
        vectorize=True)
# add the data to the existing dataset
for arr_name, arr in zip(array_names, res):
    ds[arr_name] = arr
Although still serial, apply_ufunc is around 36x faster than the loop implementation in this specific case.

However the parallelization with dask is still not implemented with multiple output like the one from scipy.stats.linregress:

In [ ]:
# I tried:

In [159]:
# - Use apply_ufunc to eliminate need for for loops
# --> works great! BUT no good way to get rid of nans and scipy pearsonr will not take nans...
betnow = bet_cp_tot[400:405,10:12,10:12] # all non-nan
skjnow = skj_cp_tot[400:405,10:12,10:12] # all non-nan
valid_values = betnow.notnull() & skjnow.notnull()
# the below two lines don't work, see here:
# https://stackoverflow.com/questions/52553925/python-xarray-remove-coordinates-with-all-missing-variables?rq=1
test = betnow.where(valid_values, drop=True) # this doesn't work
test1 = skjnow.where(valid_values, drop=True) # this doesn't work

In [160]:
testoutput = xr.apply_ufunc(stats.pearsonr, test1, test, input_core_dims=[['time'],['time']], output_core_dims=[[],[]],vectorize=True)
testoutput
# --> works great and matches #s from for loop method, but needs to have NO NANS!!! 

(<xarray.DataArray (lat: 2, lon: 2)>
 array([[0.74096692, 0.76881806],
        [0.45208595, 0.89674582]])
 Coordinates:
   * lon      (lon) float32 162.5 167.5
   * lat      (lat) float32 -2.5 2.5, <xarray.DataArray (lat: 2, lon: 2)>
 array([[0.15195936, 0.12870626],
        [0.44464372, 0.03920595]])
 Coordinates:
   * lon      (lon) float32 162.5 167.5
   * lat      (lat) float32 -2.5 2.5)

In [ ]:
[testoutput1cc, testoutput1pval]=gettempccmap_loop(test1,test,'bvss_cp_tot')
testoutput1cc

In [ ]:
testoutput1pval

In [ ]:
%%timeit
testoutput = xr.apply_ufunc(stats.pearsonr, test1, test, input_core_dims=[['time'],['time']], output_core_dims=[[],[]],vectorize=True)

In [ ]:
%%timeit
[testoutput1cc, testoutput1pval]=gettempccmap_loop(test1,test,'bvss_cp_tot')

In [ ]:
# - Convert to df to remove nans, then compute corr coeffs in df:
# --> not great method b/c can't get back to DataArray easily and can't compute p-vals easily
dsnow = xr.merge([skj_cp_tot, bet_cp_tot])
dfnow = dsnow.to_dataframe()
dfnownn = dfnow.dropna(how='any')
dfnownn.head
# following line doesn't work
#stats.pearsonr(dfnownn.groupby(level=[0,1])['skj_cp_tot'],dfnownn.groupby(level=[0,1])['bet_cp_tot'])
dfnownn.groupby(level=[0,1]).corr()

In [ ]:
# - What if we get rid of nans in the df and then convert back to xr Dataset?
# --> doesn't work, just gives nans back to have uniform grid
dsnownn = dfnownn.to_xarray()
dsnownn # --> gives nans back

In [ ]:
# - What about getting indices that are non-null and then selecting those indices?
valid_values = betnow.notnull() & skjnow.notnull()
vv = valid_values.sum(dim='time')>2
vv_stacked = vv.stack(x=['lat','lon'])
vvnow = vv_stacked[vv_stacked]
skj_cp_tot.sel(lon=vvnow.lon.values[0], lat=vvnow.lat.values[0])

In [ ]:
# - What about getting indices that are non-null and then selecting those indices?
# --> but how do I get the spatial structure back and correl over time dimension?
valid_values = betnow.notnull() & skjnow.notnull()
vv = valid_values.sum(dim='time')>2
vv_stacked = vv.stack(x=['lat','lon'])
vvnow = vv_stacked[vv_stacked]
skj_cp_tot.sel(lon=vvnow.lon.values[0], lat=vvnow.lat.values[0])

In [ ]:
valid_values = betnow.notnull() & skjnow.notnull()
vv_stacked = valid_values.stack(x=['lat','lon','time'])
vvnow = vv_stacked[vv_stacked]
skj_cp_tot.sel(lon=vvnow.lon.values[0],lat=vvnow.lat.values[0])

In [ ]:
skj_cp_tot_stacked = skj_cp_tot.stack(x=['lat','lon','time'])
skj_cp_tot_stacked[vv_stacked]

In [ ]:
test4 = skj_cp_tot.sel(time = vvnow.time.values, lat = vvnow.lat.values, lon=vvnow.lon.values)